In [1]:
# importando librerias
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import collections
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost
import math as mt
import warnings

# Ploting styles
# styles: 'fivethirtyeight', 'classic', 'ggplot', 'seaborn-notebook'
# styles: 'seaborn-poster', 'bmh', 'grayscale', 'seaborn-whitegrid'
matplotlib.style.use('bmh')
%matplotlib inline
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#print(plt.style.available)
warnings.filterwarnings("ignore")

In [7]:
#propios utilitarios
from utils import *
data_train = pd.read_csv(
    'data/01dataBaseTrainTrxRec.csv', **set_parameter_csv
).sort_values(by=['codCliente','codEstab'])
data_train.head()

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto
25041,2017-05-24 00:00:00,1,105.0,39,1,147.0,1,0.084590
522851,2017-03-07 00:00:00,1,105.0,39,1,147.0,1,0.098481
743922,2017-08-20 00:00:00,1,105.0,39,1,147.0,1,0.018126
1272804,2017-05-13 00:00:00,1,105.0,39,1,147.0,1,0.018183
816568,2016-12-22 00:00:00,1,72.0,1194,1,147.0,1,0.031735


In [8]:
data_perfil = pd.read_csv(
    'data/02dataBasePerfilRec.csv', **set_parameter_csv
).sort_values(by=['codCliente'])
data_perfil.head()

,codCliente,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4
1599,1,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
25805,2,Rango6,Rango2,1,1,241.0,Rango1,Rango3,Rango1,0,Rango6,Rango1,Rango1,Rango1
700,3,Rango6,Rango5,0,1,253.0,Rango1,Rango2,Rango2,0,Rango6,Rango2,Rango1,Rango2
19987,4,Rango6,Rango1,1,0,212.0,Rango1,Rango1,Rango2,0,Rango1,Rango1,Rango1,Rango1
29861,5,Rango6,Rango1,1,1,271.0,Rango1,Rango1,Rango3,1,Rango2,Rango1,Rango5,Rango1


In [9]:
data_train.shape, data_perfil.shape, data_train['codCliente'].unique().shape, data_train['codEstab'].unique().shape


((1591617, 8), (30000, 14), (30000,), (74339,))

In [10]:
data_tot = pd.merge(data_train, data_perfil, on='codCliente')
data_tot.head()

,fechaOper,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ctdTrx,ratingMonto,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4
0,2017-05-24 00:00:00,1,105.0,39,1,147.0,1,0.084590,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
1,2017-03-07 00:00:00,1,105.0,39,1,147.0,1,0.098481,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
2,2017-08-20 00:00:00,1,105.0,39,1,147.0,1,0.018126,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
3,2017-05-13 00:00:00,1,105.0,39,1,147.0,1,0.018183,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
4,2016-12-22 00:00:00,1,72.0,1194,1,147.0,1,0.031735,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1


In [13]:
# Eliminacion de columnas fecha y Ctdrx
data_tot.drop(['fechaOper', 'ctdTrx'], axis=1, inplace=True)
data_tot.shape, data_tot['codCliente'].unique().shape, data_tot.columns

((1591617, 19),
 (30000,),
 Index(['codCliente', 'codGiro', 'codEstab', 'flagLimaProvEstab', 'ubigeoEstab',
        'ratingMonto', 'rangoEdad', 'rangoIngreso', 'flagGenero',
        'flagLimaProvCliente', 'ubigeoCliente', 'rangoCtdProdAct',
        'rangoCtdProdPas', 'rangoCtdProdSeg', 'flagBxi', 'saldoTcEntidad1',
        'saldoTcEntidad2', 'saldoTcEntidad3', 'saldoTcEntidad4'],
       dtype='object'))

In [14]:
data_tot['ratingMonto'].describe()

count    1.591617e+06
mean     1.318383e-02
std      3.236438e-02
min      2.984905e-06
25%      1.598366e-03
50%      4.199563e-03
75%      1.152572e-02
max      1.000000e+00
Name: ratingMonto, dtype: float64

In [15]:
#Carga de CSV a entregar soluciones
data_base = pd.read_csv('data/03dataBaseTestRec.csv')
data_base.head()

,codClienteCodEstab
0,100001675
1,1000217838
2,1000219843
3,1000231391
4,100032061


In [16]:
data_pred = pd.read_csv('data/05dataBaseTestKeyRec.csv').sort_values(by=['codCliente', 'codEstab'])
data_pred.head()

,codCliente,codEstab
28161,1,39
244120,1,3843
40138,1,6314
407005,1,12767
378968,1,26260


In [17]:
data_base.shape, data_pred.shape, data_base['codClienteCodEstab'].unique().shape

((467203, 1), (467203, 2), (467203,))

In [15]:
print(data_train.shape)
combine_client_estab = list((data_train["codCliente"].map(str) + data_train["codEstab"].map(str)).unique())
print(len(combine_client_estab))
set_client_estab = set(combine_client_estab)
print(len(set_client_estab))

(1591617, 8)
901865
901865


In [16]:
set_est_client_pred = set(list(BaseSub['codClienteCodEstab'].map(str).unique()))
len(set_est_client_pred)

467203

In [17]:
len(set_client_estab & set_est_client_pred), len(set_client_estab | set_est_client_pred)

(194237, 1174831)

In [18]:
len(set_est_client_pred - set_client_estab)

272966

## Analisis de su eliminacion de duplicados

In [20]:
data_estab = data_train.drop(['codCliente','ratingMonto', 'fechaOper', 'ctdTrx'], axis=1)
data_estab = data_estab.drop_duplicates()
print(data_estab.shape, data_estab.columns)
data_estab.head()

(74339, 4) Index(['codGiro', 'codEstab', 'flagLimaProvEstab', 'ubigeoEstab'], dtype='object')


,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab
25041,105.0,39,1,147.0
816568,72.0,1194,1,147.0
512607,66.0,1911,1,136.0
867506,NaN,2399,0,NaN
1331328,110.0,2425,1,170.0


In [24]:
data_pred.shape, data_perfil.shape

((467203, 2), (30000, 14))

In [25]:
test_tot = pd.merge(data_pred, data_perfil, how='left', on=['codCliente'])
test_tot.shape

(467203, 15)

In [26]:
test_tot = pd.merge(test_tot, data_estab, how='left', on=['codEstab'])
test_tot.shape

(467203, 18)

In [27]:
test_tot.dtypes

codCliente               int64
codEstab                 int64
rangoEdad               object
rangoIngreso            object
flagGenero               int64
flagLimaProvCliente      int64
ubigeoCliente          float64
rangoCtdProdAct         object
rangoCtdProdPas         object
rangoCtdProdSeg         object
flagBxi                  int64
saldoTcEntidad1         object
saldoTcEntidad2         object
saldoTcEntidad3         object
saldoTcEntidad4         object
codGiro                float64
flagLimaProvEstab      float64
ubigeoEstab            float64
dtype: object

In [29]:
data_tot.head()

,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,ratingMonto,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4
0,1,105.0,39,1,147.0,0.084590,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
1,1,105.0,39,1,147.0,0.098481,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
2,1,105.0,39,1,147.0,0.018126,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
3,1,105.0,39,1,147.0,0.018183,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1
4,1,72.0,1194,1,147.0,0.031735,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1


In [31]:
test_tot.head()

,codCliente,codEstab,rangoEdad,rangoIngreso,flagGenero,flagLimaProvCliente,ubigeoCliente,rangoCtdProdAct,rangoCtdProdPas,rangoCtdProdSeg,flagBxi,saldoTcEntidad1,saldoTcEntidad2,saldoTcEntidad3,saldoTcEntidad4,codGiro,flagLimaProvEstab,ubigeoEstab
0,1,39,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,105.0,1.0,147.0
1,1,3843,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,50.0,1.0,170.0
2,1,6314,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,NaN,0.0,NaN
3,1,12767,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,NaN,NaN,NaN
4,1,26260,Rango6,Rango5,1,1,246.0,Rango1,Rango2,Rango1,1,Rango1,Rango1,Rango1,Rango1,85.0,0.0,128.0


In [32]:
data_tot.shape, test_tot.shape

((1591617, 19), (467203, 18))

In [33]:
y_train = data_tot['ratingMonto']

In [35]:
data_tot = data_tot.drop(['ratingMonto'], axis=1)
data_tot.shape

(1591617, 18)

In [38]:
test_tot = test_tot[[*list(data_tot.columns)]]
test_tot.shape

(467203, 18)

In [41]:
list(data_tot.columns) == list(test_tot.columns)

True

In [42]:
data_tot = pd.get_dummies(data_tot)
test_tot = pd.get_dummies(test_tot)
list(data_tot.columns) == list(test_tot.columns)

True

In [43]:
data_tot.head()

,codCliente,codGiro,codEstab,flagLimaProvEstab,ubigeoEstab,flagGenero,flagLimaProvCliente,ubigeoCliente,flagBxi,rangoEdad_Rango1,rangoEdad_Rango2,rangoEdad_Rango3,rangoEdad_Rango4,rangoEdad_Rango5,rangoEdad_Rango6,rangoIngreso_Rango1,rangoIngreso_Rango2,rangoIngreso_Rango3,rangoIngreso_Rango4,rangoIngreso_Rango5,rangoIngreso_Rango6,rangoCtdProdAct_Rango1,rangoCtdProdAct_Rango2,rangoCtdProdAct_Rango3,rangoCtdProdPas_Rango1,rangoCtdProdPas_Rango2,rangoCtdProdPas_Rango3,rangoCtdProdSeg_Rango1,rangoCtdProdSeg_Rango2,rangoCtdProdSeg_Rango3,rangoCtdProdSeg_Rango4,saldoTcEntidad1_Rango1,saldoTcEntidad1_Rango2,saldoTcEntidad1_Rango3,saldoTcEntidad1_Rango4,saldoTcEntidad1_Rango5,saldoTcEntidad1_Rango6,saldoTcEntidad2_Rango1,saldoTcEntidad2_Rango2,saldoTcEntidad2_Rango3,saldoTcEntidad2_Rango4,saldoTcEntidad2_Rango5,saldoTcEntidad3_Rango1,saldoTcEntidad3_Rango2,saldoTcEntidad3_Rango3,saldoTcEntidad3_Rango4,saldoTcEntidad3_Rango5,saldoTcEntidad3_Rango6,saldoTcEntidad4_Rango1,saldoTcEntidad4_Rango2,saldoTcEntidad4_Rango3,saldoTcEntidad4_Rango4,saldoTcEntidad4_Rango5,saldoTcEntidad4_Rango6
0,1,105.0,39,1,147.0,1,1,246.0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1,1,105.0,39,1,147.0,1,1,246.0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
2,1,105.0,39,1,147.0,1,1,246.0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
3,1,105.0,39,1,147.0,1,1,246.0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,1,72.0,1194,1,147.0,1,1,246.0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [53]:
data_tot.fillna(0, inplace=True)
test_tot.fillna(0, inplace=True)
del data_tot['codCliente']
del data_tot['codEstab']
del test_tot['codCliente']
del test_tot['codEstab']
list(data_tot.columns) == list(test_tot.columns), data_tot.shape

(True, (1591617, 52))

In [58]:
from sklearn import linear_model
from sklearn import model_selection
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import KFold

foldd =  KFold(n_splits=5, random_state=2, shuffle=False)

est = linear_model.Ridge(alpha=0.1, normalize=True)
cv_result = model_selection.cross_validate(
    est, 
    X=data_tot,
    y=y_train,
    cv=foldd, 
    return_train_score=True, 
    scoring='neg_mean_squared_error'
)
est.fit(data_tot, y_train)

#Get each vaiable name with its coeficient
coef_list=sorted(zip(data_tot.columns, np.abs(est.coef_)), key=lambda tup: tup[1], reverse=True)
coef_list[:5]

[('rangoIngreso_Rango1', 0.005066716796082711),
 ('rangoIngreso_Rango6', 0.0037891407061834835),
 ('flagBxi', 0.0036720289427616235),
 ('flagLimaProvCliente', 0.0033460630731318554),
 ('rangoIngreso_Rango5', 0.003225008839301581)]

In [59]:
ridge_df = pd.DataFrame(coef_list)
ridge_df = ridge_df.sort_values(1, ascending=True)
ridge_df['ridge_ord']=ridge_df.index
ridge_df.head(10)

,0,1,ridge_ord
51,ubigeoCliente,0.000006,51
50,ubigeoEstab,0.000011,50
49,rangoEdad_Rango4,0.000017,49
48,rangoEdad_Rango6,0.000018,48
47,codGiro,0.000023,47
46,saldoTcEntidad2_Rango4,0.000047,46
45,saldoTcEntidad2_Rango3,0.000073,45
44,rangoEdad_Rango3,0.000130,44
43,rangoCtdProdPas_Rango2,0.000140,43
42,rangoEdad_Rango5,0.000144,42


In [60]:
cols_model = list(ridge_df[0][-40:])
cols_model

['flagGenero',
 'rangoCtdProdAct_Rango1',
 'rangoCtdProdAct_Rango3',
 'saldoTcEntidad4_Rango6',
 'saldoTcEntidad3_Rango1',
 'saldoTcEntidad4_Rango1',
 'rangoEdad_Rango2',
 'saldoTcEntidad4_Rango2',
 'saldoTcEntidad2_Rango2',
 'saldoTcEntidad2_Rango5',
 'rangoIngreso_Rango3',
 'rangoCtdProdPas_Rango1',
 'rangoEdad_Rango1',
 'saldoTcEntidad4_Rango5',
 'saldoTcEntidad3_Rango3',
 'saldoTcEntidad2_Rango1',
 'saldoTcEntidad4_Rango4',
 'rangoCtdProdSeg_Rango2',
 'saldoTcEntidad1_Rango3',
 'saldoTcEntidad3_Rango5',
 'rangoCtdProdSeg_Rango1',
 'saldoTcEntidad4_Rango3',
 'saldoTcEntidad3_Rango6',
 'rangoCtdProdSeg_Rango3',
 'saldoTcEntidad3_Rango2',
 'saldoTcEntidad3_Rango4',
 'rangoCtdProdSeg_Rango4',
 'saldoTcEntidad1_Rango1',
 'rangoIngreso_Rango4',
 'saldoTcEntidad1_Rango4',
 'rangoCtdProdPas_Rango3',
 'rangoIngreso_Rango2',
 'flagLimaProvEstab',
 'saldoTcEntidad1_Rango5',
 'saldoTcEntidad1_Rango6',
 'rangoIngreso_Rango5',
 'flagLimaProvCliente',
 'flagBxi',
 'rangoIngreso_Rango6',
 'rangoIn

In [63]:
import lightgbm as lgb
from sklearn.metrics import r2_score, mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [10, 20, 30],
    'n_estimators': [50, 100, 250]
}

lgb_regressor = lgb.LGBMRegressor(
    save_binary=True,
    objective='regression',
    max_bin=255,
    learning_rate=0.1,
    bagging_fraction=0.75
)

gbm_1 = GridSearchCV(lgb_regressor, param_grid, cv=foldd, n_jobs=4, scoring='neg_mean_squared_error')
gbm_1.fit(data_tot, y_train)

best_params_1 = gbm_1.best_params_
model_1 = gbm_1.best_estimator_
score_1 = gbm_1.best_score_
score_1

-0.0008965364172557307

In [64]:
np.sqrt(-1*score_1)

0.029942217974888413

In [65]:
best_params_1

{'bagging_fraction': 0.5, 'max_depth': 10, 'n_estimators': 250}

In [66]:
X_train,X_test,y_trainn,y_test = train_test_split(data_tot,y_train,test_size=0.2,random_state=1)

In [67]:
y_p_train = model_1.predict(X_train)
print('RSME',mt.sqrt(mean_squared_error(y_trainn, y_p_train)))

RSME 0.029420047642874994


In [69]:
y_p_test = model_1.predict(X_test)
print('RSME',mt.sqrt(mean_squared_error(y_test, y_p_test)))

RSME 0.02958832388783087
